In [ ]:
#pip install pandas
#pip install scikit-learn
#pip install xgboost
#pip install hyperopt

In [1]:
import pandas as pd

In [ ]:
#Load train data in parquet format
data = pd.read_parquet('/train.parquet', engine='pyarrow')
data

,Patient-Uid,Date,Incident
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2019-03-09,PRIMARY_DIAGNOSIS
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,2015-05-16,PRIMARY_DIAGNOSIS
3,a0dc94c6-1c7c-11ec-a3a0-16262ee38c7f,2018-01-30,SYMPTOM_TYPE_0
4,a0dc950b-1c7c-11ec-b6ec-16262ee38c7f,2015-04-22,DRUG_TYPE_0
8,a0dc9543-1c7c-11ec-bb63-16262ee38c7f,2016-06-18,DRUG_TYPE_1
...,...,...,...
29080886,a0ee9f75-1c7c-11ec-94c7-16262ee38c7f,2018-07-06,DRUG_TYPE_6
29080897,a0ee1284-1c7c-11ec-a3d5-16262ee38c7f,2017-12-29,DRUG_TYPE_6
29080900,a0ee9b26-1c7c-11ec-8a40-16262ee38c7f,2018-10-18,DRUG_TYPE_10
29080903,a0ee1a92-1c7c-11ec-8341-16262ee38c7f,2015-09-18,DRUG_TYPE_6


In [ ]:
#Finding the unique values in Incident columns
data['Incident'].unique()

array(['PRIMARY_DIAGNOSIS', 'SYMPTOM_TYPE_0', 'DRUG_TYPE_0',
       'DRUG_TYPE_1', 'DRUG_TYPE_2', 'TEST_TYPE_0', 'DRUG_TYPE_3',
       'DRUG_TYPE_4', 'DRUG_TYPE_5', 'DRUG_TYPE_6', 'DRUG_TYPE_8',
       'DRUG_TYPE_7', 'SYMPTOM_TYPE_1', 'DRUG_TYPE_10', 'SYMPTOM_TYPE_29',
       'SYMPTOM_TYPE_2', 'DRUG_TYPE_11', 'DRUG_TYPE_9', 'DRUG_TYPE_13',
       'SYMPTOM_TYPE_5', 'TEST_TYPE_1', 'SYMPTOM_TYPE_6', 'TEST_TYPE_2',
       'SYMPTOM_TYPE_3', 'SYMPTOM_TYPE_8', 'DRUG_TYPE_14', 'DRUG_TYPE_12',
       'SYMPTOM_TYPE_9', 'SYMPTOM_TYPE_10', 'SYMPTOM_TYPE_7',
       'SYMPTOM_TYPE_11', 'TEST_TYPE_3', 'DRUG_TYPE_15', 'SYMPTOM_TYPE_4',
       'SYMPTOM_TYPE_14', 'SYMPTOM_TYPE_13', 'SYMPTOM_TYPE_16',
       'SYMPTOM_TYPE_17', 'SYMPTOM_TYPE_15', 'SYMPTOM_TYPE_18',
       'SYMPTOM_TYPE_12', 'SYMPTOM_TYPE_20', 'SYMPTOM_TYPE_21',
       'DRUG_TYPE_17', 'SYMPTOM_TYPE_22', 'TEST_TYPE_4',
       'SYMPTOM_TYPE_23', 'DRUG_TYPE_16', 'TEST_TYPE_5',
       'SYMPTOM_TYPE_19', 'SYMPTOM_TYPE_24', 'SYMPTOM_TYPE_25',
   

In [ ]:
import datetime

In [ ]:
#Taking the ids of patients with target drug in incident
patient_w_target = data[data['Incident']=='TARGET DRUG']['Patient-Uid'].unique().tolist()

In [ ]:
# Create an empty array, positive and negative set.
#Sort the column 'Patient_Uid' based on the date
# Store data containing incident - target drug and set column target with value 1. The data will be in the positive set if the time difference with the previous incident is more than 30 days.
# Store data in negative set , where patient has Target drug in incident, and more than 30 days difference with the previous incident and set column target with value 0

positive_set = []
negative_set = []
for patient in patient_w_target:
    patient_uid = data[data['Patient-Uid']== patient]
    patien_uid_sort = patient_uid.sort_values('Date')
    temp = []
    for i in range(len(patien_uid_sort)):
        if patien_uid_sort.iloc[i].Incident=='TARGET DRUG' and i != 0 and (datetime.datetime.fromisoformat(str(patien_uid_sort.iloc[i].Date)) - datetime.datetime.fromisoformat(str(patien_uid_sort.iloc[i-1].Date))).days < 30:
            positive_set.append([patien_uid_sort.iloc[i]['Patient-Uid'],' '.join(temp), 1])
        elif i != 0 and (patien_uid_sort.iloc[i].Incident=='TARGET DRUG' and (datetime.datetime.fromisoformat(str(patien_uid_sort.iloc[i].Date)) - datetime.datetime.fromisoformat(str(patien_uid_sort.iloc[i-1].Date))).days > 30):
            negative_set.append([patien_uid_sort.iloc[i]['Patient-Uid'],' '.join(temp), 0])

        temp.append(patien_uid_sort.iloc[i].Incident)

In [ ]:
#Taking the ids of patients without target drug in incident
patient_w_o_target = data[data['Incident']!='TARGET DRUG']['Patient-Uid'].unique().tolist()

In [ ]:
# Sort the column 'Patient_Uid' based on the date
# Add data in negative set , where patient has no Target drug in incident,and set the column target with values 0.

for patient in patient_w_o_target:
    patient_uid = data[data['Patient-Uid']== patient]
    patien_uid_sort = patient_uid.sort_values('Date')
    negative_set.append([patient,' '.join(patien_uid_sort.Incident.tolist()), 0])

In [ ]:
#Create a csv file containing the positive set
train_df = pd.DataFrame(positive_set,columns = ['Patient-Uid','Incidents','Target'])
train_df.to_csv('train_positive_set')

In [ ]:
#Create a csv file containing the negative set
train_df_1 = pd.DataFrame(negative_set,columns = ['Patient-Uid','Incidents','Target'])
train_df_1.to_csv('train_negative_set')

In [ ]:
train_positive_set = pd.read_csv('/content/train_positive_set')
train_negative_set = pd.read_csv('/content/train_negative_set')

In [ ]:
#Concat the dataframes
train_dataframes = [train_positive_set,train_negative_set]
train_dataframes_conc = pd.concat(train_dataframes)

In [ ]:
#Convert the dataframes without shuffle to csv
train_dataframes_conc.to_csv('train_dataframes_w_o_shuffle')


In [ ]:
#Load test data in parquet format
test = pd.read_parquet('/test.parquet', engine='pyarrow')
test

,Patient-Uid,Date,Incident
0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2016-12-08,SYMPTOM_TYPE_0
1,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-10-17,DRUG_TYPE_0
2,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-12-01,DRUG_TYPE_2
3,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-12-05,DRUG_TYPE_1
4,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-11-04,SYMPTOM_TYPE_0
...,...,...,...
1372854,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-05-11,DRUG_TYPE_13
1372856,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2018-08-22,DRUG_TYPE_2
1372857,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-02-04,DRUG_TYPE_2
1372858,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-09-25,DRUG_TYPE_8


In [ ]:
#Sort the data based on date, and store it in test set.
test_patient_w_o_target = test['Patient-Uid'].unique().tolist()
test_set = []
for patient in test_patient_w_o_target:
    patient_uid = test[test['Patient-Uid']== patient]
    patien_uid_sort = patient_uid.sort_values('Date')
    test_set.append([patient,' '.join(patien_uid_sort.Incident.tolist())])

In [ ]:
test_df = pd.DataFrame(test_set,columns = ['Patient-Uid','Incidents'])

In [ ]:
test_df.to_csv('test_csv')

In [ ]:
#Shuffle the training data
train = pd.read_csv('/content/train_dataframes_w_o_shuffle')
train_with_shuffle = train.sample(frac=1).reset_index(drop=True)
train_with_shuffle

In [ ]:
# train_with_shuffle.to_csv('train_csv')

In [21]:
testing_set = pd.read_csv('/content/train_csv')

In [25]:
#feature selection
Features = ['Incidents']
X = testing_set[Features].copy()

X['Incidents'] = X['Incidents'].astype('category')
X

,Incidents
0,TEST_TYPE_1 DRUG_TYPE_6 DRUG_TYPE_1 TEST_TYPE_...
1,SYMPTOM_TYPE_6 DRUG_TYPE_7 DRUG_TYPE_2 DRUG_TY...
2,DRUG_TYPE_7 DRUG_TYPE_6 DRUG_TYPE_6 DRUG_TYPE_...
3,DRUG_TYPE_7 DRUG_TYPE_1 DRUG_TYPE_1 DRUG_TYPE_...
4,TEST_TYPE_1 SYMPTOM_TYPE_7 SYMPTOM_TYPE_7 DRUG...
...,...
93943,DRUG_TYPE_7 SYMPTOM_TYPE_6 DRUG_TYPE_2 DRUG_TY...
93944,DRUG_TYPE_6 DRUG_TYPE_11 PRIMARY_DIAGNOSIS DRU...
93945,SYMPTOM_TYPE_14 DRUG_TYPE_6 SYMPTOM_TYPE_14 SY...
93946,DRUG_TYPE_6 DRUG_TYPE_7 PRIMARY_DIAGNOSIS DRUG...


In [26]:
y = testing_set['Target']
y

0        1
1        1
2        1
3        1
4        1
        ..
93943    1
93944    0
93945    0
93946    0
93947    1
Name: Target, Length: 93948, dtype: int64

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Use the training data for training and validation

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)

In [31]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

In [32]:
#Create a model and validate it using the train data

xgb_model = XGBClassifier(enable_categorical=True)
param_grid = { 'max_depth': [4,5,6] , 'min_child_weight':[4,5,6] ,'learning_rate': [0.05,0.1,0.5] ,'n_estimators': [20,50,100] }
model = GridSearchCV(xgb_model, param_grid)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(y_pred)
print()
print(accuracy_score(y_pred, y_test))

[1 1 1 ... 1 1 1]

0.6279218290969473


In [33]:
#Calculate f1_score

f1_score(y_test, y_pred, average='micro')

0.6279218290969473

In [70]:
test_set = pd.read_csv('/content/test_csv')
test_set

,Unnamed: 0,Patient-Uid,Incidents
0,0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,DRUG_TYPE_7 DRUG_TYPE_11 DRUG_TYPE_0 SYMPTOM_T...
1,1,a0f9e9f9-1c7c-11ec-b565-16262ee38c7f,PRIMARY_DIAGNOSIS DRUG_TYPE_2 DRUG_TYPE_6 DRUG...
2,2,a0f9ea43-1c7c-11ec-aa10-16262ee38c7f,DRUG_TYPE_1 DRUG_TYPE_6 DRUG_TYPE_1 DRUG_TYPE_...
3,3,a0f9ea7c-1c7c-11ec-af15-16262ee38c7f,DRUG_TYPE_6 DRUG_TYPE_3 PRIMARY_DIAGNOSIS TEST...
4,4,a0f9eab1-1c7c-11ec-a732-16262ee38c7f,DRUG_TYPE_2 DRUG_TYPE_2 DRUG_TYPE_6 PRIMARY_DI...
...,...,...,...
11477,11477,a102720c-1c7c-11ec-bd9a-16262ee38c7f,DRUG_TYPE_7 DRUG_TYPE_6 DRUG_TYPE_0 DRUG_TYPE_...
11478,11478,a102723c-1c7c-11ec-9f80-16262ee38c7f,DRUG_TYPE_1 DRUG_TYPE_1 DRUG_TYPE_1 DRUG_TYPE_...
11479,11479,a102726b-1c7c-11ec-bfbf-16262ee38c7f,DRUG_TYPE_6 DRUG_TYPE_6 DRUG_TYPE_6 DRUG_TYPE_...
11480,11480,a102729b-1c7c-11ec-86ba-16262ee38c7f,DRUG_TYPE_2 DRUG_TYPE_7 DRUG_TYPE_7 DRUG_TYPE_...


In [71]:
features = ['Incidents']
test_feature = test_set[features].copy()
test_feature['Incidents'] = test_feature['Incidents'].astype('category')

In [72]:
output = model.predict(test_feature)
output

array([1, 1, 1, ..., 1, 1, 1])

In [73]:
#Merge the output to the test data, and save in csv format
test_set['label'] = output
Drop_columns = ['Unnamed: 0','Incidents']
test_set.drop(Drop_columns, axis=1, inplace=True)
test_set

,Patient-Uid,label
0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,1
1,a0f9e9f9-1c7c-11ec-b565-16262ee38c7f,1
2,a0f9ea43-1c7c-11ec-aa10-16262ee38c7f,1
3,a0f9ea7c-1c7c-11ec-af15-16262ee38c7f,1
4,a0f9eab1-1c7c-11ec-a732-16262ee38c7f,1
...,...,...
11477,a102720c-1c7c-11ec-bd9a-16262ee38c7f,1
11478,a102723c-1c7c-11ec-9f80-16262ee38c7f,1
11479,a102726b-1c7c-11ec-bfbf-16262ee38c7f,1
11480,a102729b-1c7c-11ec-86ba-16262ee38c7f,1


In [74]:
test_set.to_csv('Final_submission.csv')